In [2]:
import json
import pandas as pd
import requests
from pprint import pprint
from pandas.io.json import json_normalize  

In [52]:
#Import CSV
aqi_df = pd.read_csv("assets/sample_data/daily_aqi_by_cbsa_2019.csv")

aqi_df.head()

,CBSA,CBSA Code,Date,AQI,Category,Defining Parameter,Defining Site,Number of Sites Reporting
0,"Aberdeen, SD",10100,2019-01-03,4,Good,PM10,46-013-0003,1
1,"Aberdeen, SD",10100,2019-01-06,31,Good,PM2.5,46-013-0003,1
2,"Aberdeen, SD",10100,2019-01-09,9,Good,PM10,46-013-0003,1
3,"Aberdeen, SD",10100,2019-01-12,43,Good,PM2.5,46-013-0003,1
4,"Aberdeen, SD",10100,2019-01-15,28,Good,PM2.5,46-013-0003,1


In [53]:
#Rename cities with duplicate names to avoid merging the wrong data
aqi_df = aqi_df.replace({"Columbia, MO": "ColumbiaMO", "Salem, OH": "SalemOH", "Columbus, GA-AL" : "ColumbusGA", "Columbus, IN" : "ColumbusIN",
                                "Jackson, TN": "JacksonTN", "Jackson, WY-ID": "JacksonWY", "Miami, OK": "MiamiOK", "Salem, OH": "SalemOH",
                                "Springfield, MA": "SpringfieldMA", "Springfield, MO" : "SpringfieldMO", "Springfield, OH": "SpringfieldOH"})



In [54]:
#Split CBSA into City and State
aqi_df[['City','State']] = aqi_df.CBSA.str.split(",",expand=True)

aqi_df.head()

,CBSA,CBSA Code,Date,AQI,Category,Defining Parameter,Defining Site,Number of Sites Reporting,City,State
0,"Aberdeen, SD",10100,2019-01-03,4,Good,PM10,46-013-0003,1,Aberdeen,SD
1,"Aberdeen, SD",10100,2019-01-06,31,Good,PM2.5,46-013-0003,1,Aberdeen,SD
2,"Aberdeen, SD",10100,2019-01-09,9,Good,PM10,46-013-0003,1,Aberdeen,SD
3,"Aberdeen, SD",10100,2019-01-12,43,Good,PM2.5,46-013-0003,1,Aberdeen,SD
4,"Aberdeen, SD",10100,2019-01-15,28,Good,PM2.5,46-013-0003,1,Aberdeen,SD


In [55]:
#Rename City metropolitan areas to one main city for merging
aqi_df = aqi_df.replace({"Atlanta-Sandy Springs-Roswell":"Atlanta","Austin-Round Rock":"Austin","Baltimore-Columbia-Towson":"Baltimore",
                        "Boise City":"Boise","Boston-Cambridge-Newton":"Boston","New York-Newark-Jersey City":"New York City",                                                  "Charlotte-Concord-Gastonia":"Charlotte","Chicago-Naperville-Elgin":"Chicago","Dallas-Fort Worth-Arlington":"Dallas",
                        "Denver-Aurora-Lakewood":"Denver","Detroit-Warren-Dearborn":"Detroit","Hartford-West Hartford-East Hartford":"Hartford","Urban                          Honolulu":"Honolulu", "Houston-The Woodlands-Sugar Land":"Houston","Indianapolis-Carmel-Anderson":"Indianapolis","Las                                   Vegas-Henderson-Paradise":"Las Vegas","Little Rock-North Little Rock-Conway":"Little Rock","Los Angeles-Long Beach-Anaheim":"Los Angeles","Miami-Fort Lauderdale-West Palm Beach":"Miami","Milwaukee-Waukesha-West Allis":"Milwaukee",                     
                        "Nashville-Davidson--Murfreesboro--Franklin":"Nashville","Omaha-Council Bluffs":"Omaha","Philadelphia-Camden-Wilmington":
                        "Philadelphia","Phoenix-Mesa-Scottsdale":"Phoenix","Portland-Vancouver-Hillsboro":"Portland","Providence-Warwick":"Providence",
                        "Sacramento--Roseville--Arden-Arcade":"Sacramento","Minneapolis-St. Paul-Bloomington":"Saint Paul","San Antonio-New Braunfels":
                        "San Antonio","San Diego-Carlsbad":"San Diego","San Francisco-Oakland-Hayward":"San Francisco","San Jose-Sunnyvale-Santa Clara":
                        "San Jose","Seattle-Tacoma-Bellevue":"Seattle","Washington-Arlington-Alexandria":"Washington, D.C."})

In [68]:
#Replace state values of multiple states with one state
aqi_df = aqi_df.replace({" MA-NH":"MA"," IL-IN-WI":"IN"," NC-SC":"NC"," TN-MS-AR":"TN"," MN-WI":"MN"," NY-NJ-PA":"NY"," NE-IA":"NE"," PA-NJ-DE-MD":"PA",                                    " OR-WA":"OR"," RI-MA":"RI"," DC-VA-MD-WV":"DC"})

In [69]:
aqi_df["State"].value_counts()

 CA          12268
 TX           9244
 PA           7931
 FL           7801
 WA           6883
             ...  
 AR-OK         361
 SC-NC         356
 TX-AR         349
 IL-MO         241
 IA-IL-MO      120
Name: State, Length: 94, dtype: int64

In [6]:
# URL for JSON with cities and geo data
url = "https://aqicn.org/data-platform/covid19/airquality-covid19-cities.json"

In [24]:
#Read JSON into a dataframe
df = pd.read_json(url)

df.head()

,csvsize,data,generated
0,46726470,"{'Place': {'geo': [38.53575, 68.77905], 'featu...",2020-05-28T20:54:02+01:00
1,46726470,"{'Place': {'geo': [24.45118, 54.39696], 'featu...",2020-05-28T20:54:02+01:00
2,46726470,"{'Place': {'geo': [25.07725, 55.30927], 'featu...",2020-05-28T20:54:02+01:00
3,46726470,"{'Place': {'geo': [60.29414, 25.04099], 'featu...",2020-05-28T20:54:02+01:00
4,46726470,"{'Place': {'geo': [60.45148, 22.26869], 'featu...",2020-05-28T20:54:02+01:00


In [7]:
# Uses requests.get() to load JSON into text
response = json.loads(requests.get(url).text)

#Flattens the nested JSON of the 'data' column into its own DataFrame to get the Places geo data
geo_df = pd.io.json.json_normalize(response['data'])
#pprint(response)
geo_df.head()

,Sources,Stations,Place.geo,Place.feature,Place.name,Place.country,Place.pop
0,[{'name': 'Citizen Weather Observer Program (C...,"[{'Name': 'Olivais, Lisboa, Portugal'}, {'Name...","[38.71667, -9.13333]",PPLC,Lisbon,PT,517802
1,[{'name': 'Citizen Weather Observer Program (C...,"[{'Name': 'Santana, Santana, Portugal'}, {'Nam...","[32.66568, -16.92547]",PPLA,Funchal,PT,100847
2,[{'name': 'Citizen Weather Observer Program (C...,"[{'Name': 'Pobedy av. 287, Chelyabinsk, Russia...","[55.15402, 61.42915]",PPLA,Chelyabinsk,RU,1062919
3,[{'name': 'Citizen Weather Observer Program (C...,"[{'Name': 'Beketova str., 28, Nizhny Novgorod,...","[56.32867, 44.00205]",PPLA,Nizhniy Novgorod,RU,1284164
4,[{'name': 'Citizen Weather Observer Program (C...,"[{'Name': 'Professor Popov str., 48, Saint-Pet...","[59.93863, 30.31413]",PPLA,Saint Petersburg,RU,5028000


In [8]:
#Puts the geo data and name into a dataframe
places_df = geo_df[['Place.geo','Place.name','Place.country']]

In [9]:
places_df.head()

,Place.geo,Place.name,Place.country
0,"[38.71667, -9.13333]",Lisbon,PT
1,"[32.66568, -16.92547]",Funchal,PT
2,"[55.15402, 61.42915]",Chelyabinsk,RU
3,"[56.32867, 44.00205]",Nizhniy Novgorod,RU
4,"[59.93863, 30.31413]",Saint Petersburg,RU


In [10]:
#Rename columns for merging purposes
places_df = places_df.rename(columns = {'Place.geo':'Geo','Place.name':'City', 'Place.country':'Country'})
places_df.head()

,Geo,City,Country
0,"[38.71667, -9.13333]",Lisbon,PT
1,"[32.66568, -16.92547]",Funchal,PT
2,"[55.15402, 61.42915]",Chelyabinsk,RU
3,"[56.32867, 44.00205]",Nizhniy Novgorod,RU
4,"[59.93863, 30.31413]",Saint Petersburg,RU


In [57]:
us_df = places_df.loc[places_df["Country"] == "US"]
print(us_df["City"].value_counts())
us_df["City"].isin(aqi_df["City"]).value_counts()

Oakland             1
Jackson             1
The Bronx           1
Boise               1
Staten Island       1
Washington, D.C.    1
Honolulu            1
Providence          1
Seattle             1
Baltimore           1
Columbus            1
Queens              1
Jacksonville        1
Chicago             1
Columbia            1
Hartford            1
Memphis             1
Denver              1
Fresno              1
Salt Lake City      1
San Jose            1
Saint Paul          1
Salem               1
Boston              1
Charlotte           1
Portland            1
Fort Worth          1
Milwaukee           1
Raleigh             1
Phoenix             1
Richmond            1
Brooklyn            1
Nashville           1
Philadelphia        1
Manhattan           1
Indianapolis        1
Dallas              1
Madison             1
Tucson              1
Detroit             1
San Diego           1
Tallahassee         1
Omaha               1
Atlanta             1
Miami               1
Sacramento

True     48
False     9
Name: City, dtype: int64

In [59]:
#Replace Manhattan with New York City for the merge
us_df = us_df.replace({"Manhattan":"New York City"})

In [21]:
us_df.to_csv("us_locations.csv")

In [70]:
#Inner merge
new_df = aqi_df.merge(us_df,on="City",how="inner")

new_df.head()

,CBSA,CBSA Code,Date,AQI,Category,Defining Parameter,Defining Site,Number of Sites Reporting,City,State,Geo,Country
0,"Albuquerque, NM",10740,2019-01-01,42,Good,PM10,35-001-0029,8,Albuquerque,NM,"[35.08449, -106.65114]",US
1,"Albuquerque, NM",10740,2019-01-02,40,Good,PM2.5,35-001-1012,8,Albuquerque,NM,"[35.08449, -106.65114]",US
2,"Albuquerque, NM",10740,2019-01-03,55,Moderate,PM2.5,35-001-0029,8,Albuquerque,NM,"[35.08449, -106.65114]",US
3,"Albuquerque, NM",10740,2019-01-04,57,Moderate,PM2.5,35-001-0029,8,Albuquerque,NM,"[35.08449, -106.65114]",US
4,"Albuquerque, NM",10740,2019-01-05,66,Moderate,PM2.5,35-001-0029,8,Albuquerque,NM,"[35.08449, -106.65114]",US


In [71]:

new_df["State"].value_counts()

 CA    2190
 TX    1825
 FL    1093
 AZ     730
 WI     730
 UT     365
 OK     365
 IN     365
 AR     365
DC      365
 GA     365
 OH     365
 MD     365
 ID     365
PA      365
 CO     365
MA      365
 MI     365
OR      365
 MS     365
 TN     365
NY      365
 WA     365
 SC     365
 NC     365
MN      365
IN      365
 CT     365
 VA     365
NE      365
NC      365
RI      365
TN      365
 NM     365
 IL     360
 OR     356
Name: State, dtype: int64

In [73]:
#Store data to CSV for verification
new_df.to_csv("sample_data.csv")

In [64]:
#Split up the location data into latitude and longitude
neww_df = pd.DataFrame(new_df['Geo'].to_list(), columns =['Lat','Lng'])

neww_df

,Lat,Lng
0,35.08449,-106.65114
1,35.08449,-106.65114
2,35.08449,-106.65114
3,35.08449,-106.65114
4,35.08449,-106.65114
...,...,...
13364,-33.64651,19.44852
13365,-33.64651,19.44852
13366,-33.64651,19.44852
13367,-33.64651,19.44852
